In [1]:
# Importing dependencies

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from pathlib import Path
import pandas as pd
from keras.models import model_from_json
from pathlib import Path
from keras.preprocessing import image
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline


from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from matrix_util import *




Using TensorFlow backend.


In [2]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))



In [3]:
print(str(sess))

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3618927442451699071
]


# Cleaning up the Dataset

In [5]:
# importing CSV file containing image labels and corresponding image file names

path_to_labels="/Users/muhammadwaliji/Desktop/facial_expressions-master/data/legend.csv"

data=pd.read_csv(path_to_labels)


In [6]:
data.head()


,user.id,image,emotion
0,628,facial-expressions_2868588k.jpg,anger
1,628,facial-expressions_2868585k.jpg,surprise
2,628,facial-expressions_2868584k.jpg,disgust
3,628,facial-expressions_2868582k.jpg,fear
4,dwdii,Aaron_Eckhart_0001.jpg,neutral


In [7]:
# Lower casing all the values in the emotion column

data['emotion'] = [str(i).lower() for i in data["emotion"]]

In [8]:
# re-labelling all the columns labelled sad with sadness for consistency

data['emotion']=data['emotion'].replace(to_replace='sadness', value='sad', regex=True)


In [9]:
labels=list(data['emotion'].unique())
labels

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sad',
 'contempt']

In [10]:
# Pick 4 emotions of choice i.e. anger, neutral, happiness, sad

restricted_to_specific_emo=data.loc[data.emotion.isin(["anger","neutral","happiness","sad"])].reset_index(drop=True)


In [11]:
# Labels we are going to use

using_labels=list(restricted_to_specific_emo['emotion'].unique())
using_labels

['anger', 'neutral', 'happiness', 'sad']

# USE OPEN CV2 to generate array for all images

In [12]:
import cv2


In [13]:
# Taking the column name "image" from the dataframe labelled data and converting to list

img_file_names=list(restricted_to_specific_emo["image"])


In [14]:
# To get the array value of each file

grayscale_image_array=[]

for i in range(len(img_file_names)):
    img_path = f"/Users/muhammadwaliji/Desktop/facial_expressions-master/images/{img_file_names[i]}"
    img = cv2.imread(img_path,0)
    grayscale_image_array.append(img)
    

In [15]:
# Count the number of images for each image size

(
    pd.DataFrame.from_records(
        [img.shape for img in grayscale_image_array],
        columns=['num_rows', 'num_cols'])
    .groupby(['num_rows', 'num_cols'])
    .size()
    .rename('num_images')
    .to_frame()
    .sort_values(by="num_images", ascending=False)
).head()

,,num_images
num_rows,num_cols,
350,350,12823
37,27,3
91,73,3
99,80,2
73,58,2


In [16]:
# Taking the array values of all the images of size 350 by 350 and their corresponding emotion values and
# appending to new lists


grayscale_image_array_only_350by350=[]
emotions_only_350by350=[]


for i in range(len(grayscale_image_array)):
    img = grayscale_image_array[i]
    emotion = restricted_to_specific_emo.emotion[i]
    
    if img.shape == (350, 350):
        grayscale_image_array_only_350by350.append(img)
        emotions_only_350by350.append(emotion)

In [17]:
# Checking the size of the new lists

print(len(grayscale_image_array_only_350by350))
print(len(emotions_only_350by350))

12823
12823


In [18]:
# Changing the orientation of the image in multiple ways to create a more robust data set which will then be used 
# to train and test the model

images_reflected_horizontally=[]

images_reflected_vertically=[]

images_rotated_right=[]

images_rotated_left=[]

images_rotated_180=[]


for i in range(len(grayscale_image_array_only_350by350)):
    
    # grabs images and assigns to img
    img=grayscale_image_array_only_350by350[i]
    
    #reflects the image horizontally and appends to images_reflected_horizontally list
    images_reflected_horizontally.append(reflect_horizontally(img))
    
    #reflects the image vertically and appends to images_reflected_vertically list
    images_reflected_vertically.append(reflect_vertically(img))
    
    #rotates the image right and appends to images_rotated_right list
    images_rotated_right.append(rotate_right(img))
    
    #rotates the image left and appends to images_rotated_left list
    images_rotated_left.append(rotate_left(img))
    
    #rotates the image 180 degrees and appends to images_rotated_180 list
    images_rotated_180.append(rotate_180(img))
    

In [19]:
# this is my final list of images (X) in various orientations

final_list_images= (
    grayscale_image_array_only_350by350 + 
    images_reflected_horizontally + 
    images_reflected_vertically +
    images_rotated_right +
    images_rotated_left+
    images_rotated_180
    
)

In [20]:
# These are the emotion labels (y) corresponding to the X images

final_list_emotions= emotions_only_350by350*6


# This is done to get y and X 


In [21]:
# Converting list grayscale_image_array_only_350by350 to array and reshaping the dimensions

X = np.concatenate([
    img.reshape(1,350,350,1)
    for img in
    final_list_images
])


In [22]:
X.shape

(76938, 350, 350, 1)

In [23]:
# converting the text label into a number between 0 and 3 (because 4 labels) and converting this to 
# an array and reshaping

label_to_number_map = {
    label: idx for idx, label in enumerate(using_labels)}

emo_numbers = [
    label_to_number_map[emo] for emo in final_list_emotions
]
emo_numbers_array = np.array(emo_numbers).reshape(-1, 1)

In [24]:
# Converting the number into one hot encoding resulting in a matrix on size 12183 X 4

y = keras.utils.to_categorical(
    emo_numbers_array,
    len(using_labels))


In [25]:
# Creating a trained test split from the data

X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y)

In [36]:
model = Sequential()

model.add(Conv2D(2, (3, 3), padding='same', input_shape=(350,350,1), activation="relu"))
model.add(Conv2D(2, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(4, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(4, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(8, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(8, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(16, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dense(4, activation="softmax"))

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Print a summary of the model
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 350, 350, 2)       20        
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 348, 348, 2)       38        
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 174, 174, 2)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 174, 174, 4)       76        
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 172, 172, 4)       148       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 86, 86, 4)         0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 86, 86, 8)         296       
__________

In [39]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import accuracy_score

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)

early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, patience=6, mode='auto')

checkpointer = ModelCheckpoint('/Users/muhammadwaliji/Desktop/facial_expressions-master/weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)



In [40]:
model.fit(X_train,
          y_train,
          batch_size=32,
          epochs=14,
          validation_data=(X_test,y_test),
          shuffle=True,
          callbacks=[lr_reducer, checkpointer, early_stopper]
)


Train on 57703 samples, validate on 19235 samples
Epoch 1/14
57703/57703 [==============================] - 2818s 49ms/step - loss: 0.6108 - acc: 0.7531 - val_loss: 0.5888 - val_acc: 0.8198

Epoch 00001: val_loss improved from inf to 0.58882, saving model to /Users/muhammadwaliji/Desktop/facial_expressions-master/weights.hd5
Epoch 2/14
57703/57703 [==============================] - 2538s 44ms/step - loss: 0.4375 - acc: 0.8505 - val_loss: 0.4159 - val_acc: 0.8596

Epoch 00002: val_loss improved from 0.58882 to 0.41590, saving model to /Users/muhammadwaliji/Desktop/facial_expressions-master/weights.hd5
Epoch 3/14
57703/57703 [==============================] - 2496s 43ms/step - loss: 0.3911 - acc: 0.8700 - val_loss: 0.3941 - val_acc: 0.8657

Epoch 00003: val_loss improved from 0.41590 to 0.39411, saving model to /Users/muhammadwaliji/Desktop/facial_expressions-master/weights.hd5
Epoch 4/14
57703/57703 [==============================] - 2503s 43ms/step - loss: 0.3594 - acc: 0.8802 - val_lo

In [41]:
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)

9398

In [42]:
# Save neural network's trained weights
model.save_weights("model_weights.h5")


In [43]:
# model.layers[0].get_weights()

In [31]:
# MHW Read it Back

from keras.models import model_from_json
from pathlib import Path
from keras.preprocessing import image
import numpy as np



# Load the json file that contains the model's structure
f2 = Path("model_structure.json")
model_structure2 = f2.read_text()

# Recreate the Keras model object from the json data
model2 = model_from_json(model_structure2)

# Re-load the model's trained weights
model2.load_weights("model_weights.h5")


In [35]:
model2.layers[0].get_weights()

[array([[[[-0.05966711, -0.23504835]],
 
         [[ 0.32434824, -0.09205551]],
 
         [[ 0.1086131 ,  0.13341534]]],
 
 
        [[[ 0.3417737 ,  0.17888854]],
 
         [[-0.20828551, -0.23628773]],
 
         [[-0.04358105,  0.31944153]]],
 
 
        [[[-0.41921687,  0.43422508]],
 
         [[ 0.44078884,  0.25204074]],
 
         [[-0.18102227,  0.05787802]]]], dtype=float32),
 array([ 0.00759535, -0.00754651], dtype=float32)]

In [ ]:
from keras.models import model_from_json
from pathlib import Path
from keras.preprocessing import image
import numpy as np



# Load the json file that contains the model's structure
f = Path("model_structure.json")
model_structure = f.read_text()

# Recreate the Keras model object from the json data
model = model_from_json(model_structure)

# Re-load the model's trained weights
model.load_weights("model_weights.h5")

# Load an image file to test, resizing it to 350 X 350 pixels (as required by this model)
img = image.load_img("frog.png", target_size=(350, 350))

# Convert the image to a numpy array
image_to_test = image.img_to_array(img)

# Add a fourth dimension to the image (since Keras expects a list of images, not a single image)
list_of_images = np.expand_dims(image_to_test, axis=0)

# Make a prediction using the model
results = model.predict(list_of_images)

# Since we are only testing one image, we only need to check the first result
single_result = results[0]

# We will get a likelihood score for all 10 possible classes. Find out which class had the highest score.
most_likely_class_index = int(np.argmax(single_result))
class_likelihood = single_result[most_likely_class_index]

# Get the name of the most likely class
class_label = class_labels[most_likely_class_index]

# Print the result
print("This is image is a {} - Likelihood: {:2f}".format(class_label, class_likelihood))

In [ ]:
# 